# Multinomial Naive Bayes

Z. W. Miller - Copyright 2018

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy
%matplotlib inline
plt.style.use('seaborn')

In [2]:
import numpy as np
import sklearn
import matplotlib
import pandas as pd
import sys
libraries = (('Matplotlib', matplotlib), ('Numpy', np), ('Pandas', pd))

print("Python Version:", sys.version, '\n')
for lib in libraries:
    print('{0} Version: {1}'.format(lib[0], lib[1].__version__))

Python Version: 3.6.2 |Anaconda custom (64-bit)| (default, Sep 21 2017, 18:29:43) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)] 

Matplotlib Version: 2.0.2
Numpy Version: 1.12.1
Pandas Version: 0.20.3


In [199]:
import pandas as pd
import numpy as np
from collections import defaultdict

class multinomial_naive_bayes:
    
    def __init__(self, smoothing = 1.):
        """
        Multinomial Naive Bayes builds it's understanding of the data by
        applying Bayes rule and calculating the conditional probability of
        being a class based on a probabilistic understanding of how the 
        class has behaved before. We calculate conditional probabilities
        . 
        ---
        Inputs:
        smoothing: the Laplace smoothing factor overcome the problem of multiplying
        a 0 probability, that causes the total probability to be 0.
        """
        self._prob_by_class = defaultdict(float)
        self._cond_probs = defaultdict(lambda: defaultdict(float))
        self._log_prob_by_class = defaultdict(float)
        self._log_cond_probs = defaultdict(lambda: defaultdict(float))
        self._denominators = defaultdict(int)
        self._data_cols = None
        self._smoothing = smoothing
    
    def fit(self, X, y):
        """
        For each class, we find out what percentage of the data is that class.
        We then filter the data so only the rows that are that class remain,
        and then go column by column - calculating what of total counts in the
        class come from that feature. We store all of these values to be used later 
        for predictions. We also store the log of these values for later prediction.
        ---
        Input: X, data (array/DataFrame)
        y, targets (array/Series)
        """
        X = self.convert_to_array(X)
        y = self.pandas_to_numpy(y)
        self._data_cols = X.shape[1]
       
        self._classes = np.unique(y)
        
        for cl in self._classes:
            filtered_targets = y[y == cl]
            filtered_data = X[y == cl]
            self._prob_by_class[cl] = len(filtered_targets)/len(y)
            self._log_prob_by_class[cl] = np.log(self._prob_by_class[cl])
            denom = np.sum(filtered_data)
            self._denominators[cl] = denom
            for col in range(self._data_cols):
                sum_of_column = np.sum(filtered_data.T[col])
                #smoothing applied here so we never get a zero probability
                self._cond_probs[cl][col] = (sum_of_column+self._smoothing)/(denom+self._smoothing) 
                self._log_cond_probs[cl][col] = np.log(self._cond_probs[cl][col])
                    
            self._largest_denom = max(self._denominators.values())
                
    def predict(self, X):
        """
        Wrapper to return only the class of the prediction
        ---
        Input: X, data (array/dataframe)
        """
        return self._predict(X, mode="predict")
    
    def predict_proba(self, X):
        """
        Wrapper to return probability of each class of the prediction
        ---
        Input: X, data (array/dataframe)
        """
        return self._predict(X, mode="predict_proba")
    
    def predict_log_proba(self, X):
        """
        Wrapper to return log of the probability of each class of 
        the prediction.
        ---
        Input: X, data (array/dataframe)
        """
        return self._predict(X, mode="predict_log_proba")
    
    def _predict(self, X, mode="predict"):
        """
        For each data point, we go through and calculate the probability
        of it being each class. We do so by using the probability of
        seeing each feature/class and multiplying that by the number
        of times we see that feature, then combining them together with 
        the class probability. We work in the log space to fight against
        combining too many really small or large values and under/over 
        flowing Python's memory capabilities for a float. Depending on the mode
        we return either the prediction, the probabilities for each class,
        or the log of the probabilities for each class.
        ---
        Inputs: X, data (array/DataFrame)
        mode: type of prediction to return, defaults to single prediction mode
        """
        X = self.convert_to_array(X)
        results = []
        for row in X:
            beliefs = []
            for cl in self._classes:
                prob_for_class = self._log_prob_by_class[cl]
                for col in range(self._data_cols):
                    val = row[col]
                    p = self._log_cond_probs[cl][col]
                    prob_for_class += val*p
                beliefs.append([cl, prob_for_class])
            
            if mode == "predict_log_proba":
                _, log_probs = zip(*beliefs)
                results.append(log_probs)
            
            elif mode == "predict_proba":
                _, probs = zip(*beliefs)
                unlog_probs = np.exp(probs)
                normed_probs = unlog_probs/np.sum(unlog_probs)
                results.append(normed_probs)
            
            else:
                sort_beliefs = sorted(beliefs, key=lambda x: x[1], reverse=True)
                results.append(sort_beliefs[0][0])
        
        return np.array(results).reshape(-1,1)
    
    def score(self, X, y):
        """
        Uses the predict method to measure the accuracy of the model.
        ---
        In: X (list or array), feature matrix; y (list or array) labels
        Out: accuracy (float)
        """
        pred = self.predict(X)
        correct = 0
        for i,j in zip(y,pred):
            if i == j:
                correct+=1
        return float(correct)/float(len(y))
      
    def pandas_to_numpy(self, x):
        """
        Checks if the input is a Dataframe or series, converts to numpy matrix for
        calculation purposes.
        ---
        Input: X (array, dataframe, or series)
        Output: X (array)
        """
        if type(x) == type(pd.DataFrame()) or type(x) == type(pd.Series()):
            return x.as_matrix()
        if type(x) == type(np.array([1,2])):
            return x
        return np.array(x) 
    
    def handle_1d_data(self,x):
        """
        Converts 1 dimensional data into a series of rows with 1 columns
        instead of 1 row with many columns.
        """
        if x.ndim == 1:
            x = x.reshape(-1,1)
        return x
    
    def convert_to_array(self, x):
        """
        Takes in an input and converts it to a numpy array
        and then checks if it needs to be reshaped for us
        to use it properly
        """
        x = self.pandas_to_numpy(x)
        x = self.handle_1d_data(x)
        return x

### Let's test it!

Let's generate some data to test with. We'll use the example of senators voting on 4 different issues (only 3 of which are relevant) and then trying to predict which party the senator is from.

In [200]:
def get_data():
    votes = [0,1]
    senators = np.random.choice(votes, replace=True, size=(100,4))
    df = pd.DataFrame(senators, columns=['vote1','vote2','vote3','vote4'])
    
    def calculate_party(row):
        x = row['vote1']
        y = row['vote2']
        z = row['vote3']

        party = 0.7*x + 0.5*y - z + np.random.normal(0,0.3)
        if party > 0.1:
            return 'Dem'
        elif party > 0.01:
            return 'Ind'
        else:
            return 'Rep'
    
    df['party'] = df.apply(calculate_party,axis=1)
    print(df.party.value_counts())
    return df.iloc[:,:-1],df.iloc[:,-1]
    

In [201]:
X, y = get_data()

Dem    51
Rep    48
Ind     1
Name: party, dtype: int64


In [202]:
nb = multinomial_naive_bayes()
nb.fit(X.iloc[:90],y.iloc[:90])

Let's look at the probability of voting YES on each issue by what party the senators in our training data were.

In [203]:
nb._cond_probs

defaultdict(<function __main__.multinomial_naive_bayes.__init__.<locals>.<lambda>>,
            {'Dem': defaultdict(float,
                         {0: 0.34444444444444444,
                          1: 0.31111111111111112,
                          2: 0.066666666666666666,
                          3: 0.31111111111111112}),
             'Ind': defaultdict(float,
                         {0: 0.66666666666666663,
                          1: 0.33333333333333331,
                          2: 0.66666666666666663,
                          3: 0.33333333333333331}),
             'Rep': defaultdict(float,
                         {0: 0.10465116279069768,
                          1: 0.20930232558139536,
                          2: 0.45348837209302323,
                          3: 0.26744186046511625})})

Now we can predict!

In [204]:
nb.predict(X.iloc[0:2])

array([['Dem'],
       ['Rep']], 
      dtype='<U3')

In [205]:
nb.predict_proba(X.iloc[0:2])

array([[ 0.52534187],
       [ 0.01279242],
       [ 0.46186571],
       [ 0.16880973],
       [ 0.04110627],
       [ 0.790084  ]])

In [206]:
nb.predict_log_proba(X.iloc[0:2])

array([[-1.8832252 ],
       [-5.59842196],
       [-2.01200026],
       [-4.5912754 ],
       [-6.00388707],
       [-3.04790837]])

We have an accuracy of 80%, which is the same as SkLearn's accuracy!

In [207]:
nb.score(X.iloc[90:],y.iloc[90:])

0.9

In [208]:
from sklearn.naive_bayes import MultinomialNB

nb_sk = MultinomialNB()
nb_sk.fit(X.iloc[:90],y.iloc[:90])
nb_sk.score(X.iloc[90:],y.iloc[90:])

0.90000000000000002

In [209]:
nb._cond_probs

defaultdict(<function __main__.multinomial_naive_bayes.__init__.<locals>.<lambda>>,
            {'Dem': defaultdict(float,
                         {0: 0.34444444444444444,
                          1: 0.31111111111111112,
                          2: 0.066666666666666666,
                          3: 0.31111111111111112}),
             'Ind': defaultdict(float,
                         {0: 0.66666666666666663,
                          1: 0.33333333333333331,
                          2: 0.66666666666666663,
                          3: 0.33333333333333331}),
             'Rep': defaultdict(float,
                         {0: 0.10465116279069768,
                          1: 0.20930232558139536,
                          2: 0.45348837209302323,
                          3: 0.26744186046511625})})

In [210]:
np.exp(nb_sk.feature_log_prob_)

array([[ 0.33333333,  0.30107527,  0.06451613,  0.30107527],
       [ 0.33333333,  0.16666667,  0.33333333,  0.16666667],
       [ 0.1011236 ,  0.20224719,  0.43820225,  0.25842697]])

# Test on Text data

In [211]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets

categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball']
ng_train = datasets.fetch_20newsgroups(subset='train', 
                                       categories=categories, 
                                       remove=('headers', 
                                               'footers', 'quotes'))

In [221]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_features=200,
                                   stop_words='english', 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6,
                                   min_df = 3)

X = count_vectorizer.fit_transform(ng_train.data)

In [222]:
X = X.todense()

In [223]:
nb = multinomial_naive_bayes()
nb.fit(X, ng_train.target)

In [224]:
nb.predict(X[:10])

array([[2],
       [0],
       [0],
       [0],
       [2],
       [0],
       [2],
       [1],
       [2],
       [2]])

In [225]:
for i,j in zip(nb.predict(X[:20]),ng_train.target[:20]):
    print(i,j)

[2] 2
[0] 2
[0] 0
[0] 0
[2] 2
[0] 0
[2] 2
[1] 1
[2] 1
[2] 2
[2] 0
[1] 1
[0] 0
[0] 0
[1] 1
[1] 1
[2] 2
[0] 0
[1] 1
[1] 1


In [226]:
nb._cond_probs

defaultdict(<function __main__.multinomial_naive_bayes.__init__.<locals>.<lambda>>,
            {0: defaultdict(float,
                         {0: 0.0028328611898016999,
                          1: 0.0054703526423756964,
                          2: 0.0011722184233662206,
                          3: 0.0053726677737618439,
                          4: 0.014555045423463905,
                          5: 0.0034189704014848101,
                          6: 0.019536973722770343,
                          7: 0.009866171729999023,
                          8: 0.015922633584057828,
                          9: 0.00087916381752466545,
                          10: 0.00048842434306925855,
                          11: 0.00400507961316792,
                          12: 0.00019536973722770342,
                          13: 0.0056657223796033997,
                          14: 0.009866171729999023,
                          15: 0.01602031845267168,
                          16: 0.00332128553287095

In [227]:
nb.score(X[:100],ng_train.target[:100])

0.85

In [228]:
from sklearn.naive_bayes import MultinomialNB

nb_sk = MultinomialNB()
nb_sk.fit(X, ng_train.target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [229]:
nb_sk.score(X[:100], ng_train.target[:100])

0.85999999999999999